In [21]:
#Importando bibliotecas
import pandas as pd
import numpy as np
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pickle

In [22]:
dataSetBruto = pd.read_csv("https://raw.githubusercontent.com/gilvandrocesardemedeiros/Meteorology_replace-missing-values/master/DadosDiarios/aracajudiario.csv",
                      sep=';',skiprows=16)

In [23]:
#Transformando data em variável do tipo datetime
dataSetBruto["Data"] = pd.to_datetime(dataSetBruto["Data"], format = "%d/%m/%Y")

In [24]:
dataSetBruto.describe()

,Estacao,Hora,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Unnamed: 11
count,22441.0,22441.000000,11198.000000,11210.000000,10076.000000,10767.000000,6743.000000,11071.000000,11196.000000,9385.000000,0.0
mean,83096.0,599.492001,3.344731,29.388662,23.116485,7.837606,3.931915,26.381604,76.868502,2.762278,NaN
std,0.0,600.013154,9.240869,1.402776,1.429518,3.128315,1.309769,1.263401,5.708472,1.094090,NaN
min,83096.0,0.000000,0.000000,23.900000,11.000000,0.000000,0.000000,19.600000,44.250000,0.000000,NaN
25%,83096.0,0.000000,0.000000,28.300000,22.200000,6.200000,3.100000,25.520000,73.250000,2.200000,NaN
50%,83096.0,0.000000,0.000000,29.500000,23.300000,8.900000,4.000000,26.520000,76.500000,2.733333,NaN
75%,83096.0,1200.000000,2.500000,30.400000,24.100000,10.200000,4.700000,27.320000,80.000000,3.400000,NaN
max,83096.0,1200.000000,157.000000,35.300000,27.100000,12.500000,18.100000,29.920000,100.000000,6.900000,NaN


In [25]:
#Verificando dados
dataSetBruto.head()

,Estacao,Data,Hora,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media,Unnamed: 11
0,83096,1980-01-01,0,NaN,29.0,NaN,4.1,2.2,26.34,75.5,4.033333,NaN
1,83096,1980-01-01,1200,0.0,NaN,22.5,NaN,NaN,NaN,NaN,NaN,NaN
2,83096,1980-01-02,0,NaN,29.1,NaN,3.9,3.0,26.90,81.5,4.333333,NaN
3,83096,1980-01-02,1200,0.0,NaN,24.3,NaN,NaN,NaN,NaN,NaN,NaN
4,83096,1980-01-03,0,NaN,29.0,NaN,0.3,2.5,26.94,86.5,4.400000,NaN


In [26]:
dataInicial = '1980-01-01'
dataFinal = '2017-12-31'
tempo = pd.date_range(dataInicial, dataFinal)

In [27]:
dataSetBruto = dataSetBruto.set_index("Data")

In [28]:
#Separando os dados em dois dataframes, um para as 00:00 h e outro para as 12:00 h
dataSetBruto00, dataSetBruto12 = dataSetBruto[dataSetBruto["Hora"] == 0], dataSetBruto[dataSetBruto["Hora"] == 1200]

In [29]:
#Descartando a coluna "Hora"
dataSetBruto00, dataSetBruto12 = dataSetBruto00.drop(columns = ["Hora", "Estacao", "Unnamed: 11"]), dataSetBruto12.drop(columns = ["Hora", "Estacao", "Unnamed: 11"])

In [30]:
#Verificando dataframe para os dados disponibilizados às 00:00 h
dataSetBruto00.head()

,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media
Data,,,,,,,,
1980-01-01,NaN,29.0,NaN,4.1,2.2,26.34,75.50,4.033333
1980-01-02,NaN,29.1,NaN,3.9,3.0,26.90,81.50,4.333333
1980-01-03,NaN,29.0,NaN,0.3,2.5,26.94,86.50,4.400000
1980-01-04,NaN,28.9,NaN,3.0,1.8,25.36,87.50,3.566667
1980-01-05,NaN,28.8,NaN,1.2,1.5,26.02,83.25,2.166667


In [31]:
#Verificando dataframe para os dados disponibilizados às 12:00 h
dataSetBruto12.head()

,Precipitacao,TempMaxima,TempMinima,Insolacao,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media
Data,,,,,,,,
1980-01-01,0.0,NaN,22.5,NaN,NaN,NaN,NaN,NaN
1980-01-02,0.0,NaN,24.3,NaN,NaN,NaN,NaN,NaN
1980-01-03,0.0,NaN,25.2,NaN,NaN,NaN,NaN,NaN
1980-01-04,12.2,NaN,25.1,NaN,NaN,NaN,NaN,NaN
1980-01-05,8.2,NaN,21.9,NaN,NaN,NaN,NaN,NaN


In [32]:
dataSetProv = pd.DataFrame()
dataSetProv["Data"] = tempo

In [44]:
#Juntando os dados em um mesmo dataframe (no caso, o dataSet00)
for i in dataSetProv["Data"].index:
    dataAux = dataSetProv.loc[i]
    try:
        dataSetProv[""] = dataSetBruto00
    except:
        print("Erro em " + str(dataAux))
        '''    dataSetProv[""]
    aracajuData00["Precipitacao"].loc[i] = aracajuData12["Precipitacao"].loc[i]
    aracajuData00["TempMinima"].loc[i] = aracajuData12["TempMinima"].loc[i]
  except:
    print("Data " + str(i.day) + "/" + str(i.month) + "/" + str(i.year) + " Não encontrada!")
    '''

TypeError: cannot do index indexing on <class 'pandas.core.indexes.datetimes.DatetimeIndex'> with these indexers [1] of <class 'int'>

In [ ]:
#Atribuindo à variável dataSet o DataFrame atualizado
aracajuData = aracajuData00
#Verificando DataFrame
aracajuData.describe()

In [ ]:
aracajuData["Mes"] = aracajuData.index.month

In [ ]:
#Transformando a coluna Data de volta em uma coluna de informações do dataSet
aracajuData = aracajuData.reset_index(drop = True)

In [ ]:
aracajuData.head()

In [ ]:
aracajuData = aracajuData.dropna(subset = ["TempMaxima"])

In [ ]:
#Separando a variável que será prevista, TempMaxima, da base de dados
tempMaxAracaju = aracajuData["TempMaxima"]
aracajuData = aracajuData.drop(columns = "TempMaxima")

In [ ]:
#Visualizando dados
aracajuData.head()

In [ ]:
tempMaxAracaju.head()

In [ ]:
colunas = aracajuData.columns

In [ ]:
#Efetuando uma transformação robusta, trabalhando com os dados entre os percentis 10 e 90
mms = MinMaxScaler(feature_range=(-1, 1))
aracajuData = mms.fit_transform(aracajuData)

In [ ]:
prov = pd.DataFrame(aracajuData)
prov.columns = colunas
prov.describe()

In [ ]:
#Trocando dados faltantes por 0
aracajuData = np.nan_to_num(aracajuData, copy = False)

In [ ]:
#Verificando dimensões do dataSet
print(aracajuData.shape)

In [ ]:
#Separando conjunto de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(aracajuData, tempMaxAracaju, test_size = 0.1, random_state = 9)

In [ ]:
#Definindo uma função para criar a MLP
def trainMLP(optimizer='adam', init='random_uniform', units = 8, camadas = 2):    
  #Criando o modelo
  mlp = Sequential()
  #Adicionando a camada de entrada
  mlp.add(Dense(activation = 'tanh', input_dim = units, units = units, kernel_initializer = init, bias_initializer='zeros'))
  #Adicionando as camadas escondidas
  for i in range(camadas):
    mlp.add(Dense(activation = 'tanh', units = units, kernel_initializer = init, bias_initializer='zeros'))
  #Adicionando a camada de saída
  mlp.add(Dense(units = 1, kernel_initializer = init, bias_initializer='zeros'))    
  # Compilando o modelo
  mlp.compile(loss='mean_squared_error', optimizer = optimizer, metrics= ['mae'])
  return mlp

In [ ]:
#Modelo de classificação do Keras
estimator = KerasRegressor(build_fn=trainMLP, epochs = 200, verbose = 2)

In [ ]:
#Criando o objeto para Cross Validation
kfold = KFold(n_splits=4, shuffle=True, random_state=5)

In [ ]:
#Parâmetros para o Grid Search
#Adicionando e alterando parâmetros neste dicionário, pode-se aumentar ou diminuir a quantidade de parâmetros a serem testados na modelagem
parameters = {'batch_size': [10],             
              'camadas': [2],
              'units': [8]}

In [ ]:
#Modelagem para buscar os melhores parâmetros
#Por limitações de poder de processamento, retirou-se o n_jobs = -1 dos parâmetros do Grid Search
grid_search = GridSearchCV(estimator = estimator, param_grid = parameters, 
                     scoring = "neg_mean_squared_error", verbose=500, cv = kfold, return_train_score = True)

In [ ]:
#Busca pelos melhores parâmetros
grid_result = grid_search.fit(X_train, Y_train)

In [ ]:
#Exibição dos melhores resultados
print("Melhor resultado: %f, usando %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
print("Resultados (Desvio padrão) {Parâmetros}")
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) %r" % (mean, stdev, param))

In [ ]:
#Gráfico de previsão para o conjunto de treino
plt.scatter(grid_search.predict(X_train), Y_train)

In [ ]:
#Gráfico de previsão para o conjunto de treino normalizado
plt.scatter(grid_search.predict(X_train) / max(grid_search.predict(X_train)), Y_train / max(Y_train))

In [ ]:
#Mostrando uma matriz de correlação das variáveis com a melhor modelagem
print(np.corrcoef(Y_train, grid_search.predict(X_train)))

In [ ]:
#Previsão com base no resultado da Grid Search
Y_pred = grid_search.predict(X_test)

In [ ]:
#Gráfico que mostra a relação entre ROP real e ROP previsto
plt.scatter(Y_test, Y_pred)

In [ ]:
#Gráfico normalizado que mostra a relação entre a chuva real e a prevista
plt.scatter(Y_test / max(Y_test), Y_pred / max(Y_pred))

In [ ]:
#Mostrando uma matriz de correlação das variáveis com a melhor modelagem
print(np.corrcoef(Y_test, Y_pred))

In [ ]:
#Erro absoluto médio entre o ROP previsto e o real
print(mean_absolute_error(Y_test, Y_pred))

In [ ]:
#Erro quadrático médio entre o ROP previsto e o real
print(mean_squared_error(Y_test, Y_pred))